In [ ]:
import os
import sys
from pathlib import Path
import psycopg2
import csv

current_dir = Path(os.getcwd())
sys.path.append(str(current_dir.parent))
import config

db_config = config.db_config

In [ ]:
def read_headers(file_path) -> list:
    with open(file_path, 'r') as f:
        headers = f.readline()
    headers_list = [header.strip() for header in headers.split(",")]
    return headers_list

In [ ]:
schema_name = "test_tables"
transaction_type_table_name = "crm_transaction_type"


transaction_type_columns_required = ["transaction_type_cd","transaction_type_nm"]
transaction_type_file_path = "data\\tr_type.csv"

print(transaction_type_columns_required)
transaction_type_columns_parsed = read_headers(transaction_type_file_path)
print(transaction_type_columns_parsed)
headers_flag = transaction_type_columns_required == transaction_type_columns_parsed
print(headers_flag)
possibility_flag =  len(transaction_type_columns_required) == len(transaction_type_columns_parsed)
print(possibility_flag)

In [ ]:

if possibility_flag:
    conn = psycopg2.connect(**db_config)
    cur = conn.cursor()

    try:
        with open(transaction_type_file_path, 'r') as f:

            cur.copy_expert(
                sql=f"""
                COPY {schema_name}.{transaction_type_table_name}
                ({", ".join(transaction_type_columns_required)})
                FROM STDIN WITH (FORMAT CSV, HEADER {headers_flag}, DELIMITER ',')
                """,
                file=f 
            )
    except Exception as e:
        print(f"Данные не загружены {str(e)}")

    conn.commit()
    cur.close()
    conn.close()

    print("Закрыли соедниене.")
else:
    print("wrong csv length")
